In [1]:
import urllib
import json
import pandas as pd

## California

In [2]:
url = 'https://data.ca.gov/api/3/action/datastore_search?resource_id=42d33765-20fd-44b8-a978-b083b7542225&limit=100000'  
fileobj = urllib.request.urlopen(url)
hospitalizations_str = fileobj.read().decode("utf-8")
hospitalizations_dict = json.loads(hospitalizations_str)
hospitalizations = pd.DataFrame(hospitalizations_dict["result"]["records"]).rename(columns={"todays_date":"date"})



In [3]:
url = 'https://data.ca.gov/api/3/action/datastore_search?resource_id=926fd08f-cc91-4828-af38-bd45de97f8c3&limit=100000'
fileobj = urllib.request.urlopen(url)
cases_str = fileobj.read().decode("utf-8")
cases_dict = json.loads(cases_str)
cases = pd.DataFrame(cases_dict["result"]["records"])




In [17]:
ca_data = pd.merge(cases, hospitalizations, how="left").\
drop(columns=["_id", "totalcountconfirmed", "totalcountdeaths"]).\
assign(state="California").\
rename(columns={"newcountconfirmed":"cases", "newcountdeaths":"deaths"})
ca_data["date"] = pd.to_datetime(ca_data["date"])


## Washington

In [30]:
nyt_data = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv").drop(columns="fips")
nyt_data["date"] = pd.to_datetime(nyt_data["date"])



In [31]:
wa_data = nyt_data[nyt_data["state"]=="Washington"]

## Oregon

In [32]:
or_data = nyt_data[nyt_data["state"]=="Oregon"]

## Aggregate

In [51]:
all_data = pd.concat([wa_data, or_data, ca_data])
first_columns = ["state", "county", "date", "cases", "deaths"]
new_col_order = first_columns + [x for x in list(all_data.columns) if x not in first_columns]
all_data = all_data[new_col_order]

/Users/lucy.li/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [53]:
all_data_agg = all_data.groupby(["state", "county", "date"]).agg(sum)

## Save to file

In [54]:
all_data.to_csv("county_data.csv", index=False)
all_data_agg.to_csv("state_data.csv", index=False)